In [1]:
import geopandas as gpd
import rasterio as rio
import fiona
import rioxarray as rxr
import re, glob
import pandas
from shapely.geometry import Polygon
import pandas as pd
import os
from rasterio.plot import show
from rasterio.merge import merge
from pathlib import Path
import numpy as np
from osgeo import gdal

In [2]:
#%conda install --channel=conda-forge gdal
# %conda update rioxarray

In [3]:
hydrosheds_firstorder = gpd.read_file("/nas/cee-water/cjgleason/wenwen/SARN_HMA/hydrosheds_firstorder.shp")
hydrosheds_firstorder = hydrosheds_firstorder.set_index(hydrosheds_firstorder.COMID)
hydrosheds_firstorder

,COMID,unitarea,slope_pl,uparea,geometry
COMID,,,,,
41000022,41000022,53.430461,0.039435,53.430461,"POLYGON ((142.56708 54.30292, 142.56792 54.302..."
41000030,41000030,56.638688,0.026666,56.638688,"POLYGON ((141.98875 53.28292, 141.99542 53.282..."
41000304,41000304,37.549159,0.007030,37.549159,"POLYGON ((141.88208 52.67375, 141.88375 52.673..."
41000202,41000202,88.947415,0.052965,88.947415,"POLYGON ((142.53542 52.16292, 142.53958 52.162..."
41000332,41000332,71.442812,0.066958,71.442812,"POLYGON ((142.34458 51.56542, 142.34792 51.565..."
...,...,...,...,...,...
49008387,49008387,41.623293,0.133969,41.623293,"POLYGON ((81.90958 32.52458, 81.91042 32.52458..."
49013157,49013157,65.608594,0.202877,65.608594,"POLYGON ((85.64208 30.77792, 85.64375 30.77792..."
49008386,49008386,64.178693,0.144355,64.178693,"POLYGON ((83.36875 32.55625, 83.37042 32.55625..."


In [4]:
intersection = gpd.read_file("/nas/cee-water/cjgleason/wenwen/SARN_HMA/overlay.shp")
print(intersection.geometry)
print(intersection.crs)

0      POLYGON ((86.35708 27.19125, 86.35708 27.19042...
1      POLYGON ((86.13708 27.20208, 86.13708 27.20125...
2      POLYGON ((86.16458 27.17042, 86.16458 27.16958...
3      POLYGON ((86.25292 27.13125, 86.25292 27.13042...
4      POLYGON ((86.49292 27.56875, 86.49292 27.56958...
                             ...                        
746    POLYGON ((83.93458 28.44125, 83.93375 28.44125...
747    POLYGON ((83.61958 28.42458, 83.61958 28.42375...
748    POLYGON ((83.60792 28.39708, 83.60792 28.39625...
749    POLYGON ((83.74292 28.42125, 83.74292 28.42042...
750    POLYGON ((83.43125 28.34042, 83.43125 28.34125...
Name: geometry, Length: 751, dtype: geometry
EPSG:4326


In [5]:
lst = intersection.COMID
COMID_lst = [*set(lst)]
#COMID_lst

In [6]:
len(COMID_lst)

637

In [7]:
COMID_lst = COMID_lst[:30]
COMID_lst 

[45017090,
 45017091,
 45035523,
 45035530,
 45039636,
 45021208,
 45017113,
 45029406,
 45039647,
 45025313,
 45029417,
 45039668,
 45025334,
 45017143,
 45029437,
 45029444,
 45025351,
 45021256,
 45029459,
 45021270,
 45025367,
 45029465,
 45029475,
 45021292,
 45017198,
 45017202,
 45021306,
 45041793,
 45027463,
 45029512]

### MAIN CODE

In [8]:
#COMID_lst = [45016003, 45021759, 45022197, 45021775]

In [9]:
#raster

In [10]:
#merged_projected.crs

In [11]:
step1 = gdal.Open('path_of_the_file.tif', gdal.GA_ReadOnly)
GT_input = step1.GetGeoTransform()

ERROR 4: path_of_the_file.tif: No such file or directory


AttributeError: 'NoneType' object has no attribute 'GetGeoTransform'

In [14]:
COMID_lst = [45039031]
for ID in COMID_lst: 
    merged = hydrosheds_firstorder[hydrosheds_firstorder.COMID==ID]
    df = intersection[intersection.COMID==ID]
    
    for i in list(df.index.values): 
        
        #get DEM crs from DEM_ID 
        dem_id = df.DEM_ID[i]
        utm = "326" + dem_id[3:5]
        
        #reproject watershed polygon 
        merged_setcrs = merged.set_crs('EPSG:4326')
        merged_projected = merged_setcrs.to_crs('EPSG:'+ utm)

        dem_path = os.path.join("/nas/cee-water/cjgleason/wenwen/SARN_HMA/EarthDEM/Order_Downloads_2023-02-22-2002/earthdem", df.DEM_ID[i], df.DEM_ID[i]+"_dem.tif")
        raster = rxr.open_rasterio(dem_path)
        
        # Use geopandas/shapely to clip raster
        clipped_raster = raster.rio.clip([merged_projected.geometry[ID]])
        
        # Save clipped raster
        clipped_raster.rio.to_raster(str(ID)+ "_" + str(dem_id)+'.tif')
        
#         dst_crs=merged_projected.crs
        
#         step1 = gdal.Open(dem_path, gdal.GA_ReadOnly)
#         GT_input = step1.GetGeoTransform()

        # with rio.open(
        #     str(ID)+ "_" + str(dem_id)+'.tif',
        #     'w',
        #     driver='GTiff',
        #     height=clipped_raster.shape[0],
        #     width=clipped_raster.shape[1],
        #     count=1,
        #     dtype=np.float32,
        #     crs=dst_crs,
        #     transform=GT_input,
        # ) as dest_file:
        #     dest_file.write(clipped_raster, 1)
        # dest_file.close()   
        
    #empty list of masked rasters to merge 
    raster_to_mosiac = []
    
    #append masked rasters to list 
    for i in list(df.index.values):
        dem_id = df.DEM_ID[i]
        folder = "/nas/cee-water/cjgleason/wenwen/SARN_HMA"
        path = os.path.join(folder, str(ID)+"_"+str(dem_id)+'.tif')
        raster = rio.open(path)
        raster_to_mosiac.append(raster)
    
    #merge list of rasters 
    mosaic, output = merge(raster_to_mosiac)
    
    #update mosaic metadata
    output_meta = raster.meta.copy()
    output_meta.update(
        {"driver": "GTiff",
            "height": mosaic.shape[1],
            "width": mosaic.shape[2],
            "transform": output,
        }
    )
    
    #write mosaic to folder 
    output_path = "/nas/cee-water/cjgleason/wenwen/SARN_HMA/mask_mosaics/"+str(ID)+".tif"
    with rio.open(output_path, "w", **output_meta) as m:
        m.write(mosaic)
        
    #delete intermediate tif files
    for i in list(df.index.values):
        dem_id = df.DEM_ID[i]
        folder = "/nas/cee-water/cjgleason/wenwen/SARN_HMA"
        path = os.path.join(folder, str(ID)+"_"+str(dem_id)+'.tif')
        os.remove(path)

In [ ]:
# original code that suddenly stopped working...

COMID_lst = [45039031]
for ID in COMID_lst: 
    merged = hydrosheds_firstorder[hydrosheds_firstorder.COMID==ID]
    df = intersection[intersection.COMID==ID]
    
    for i in list(df.index.values): 
        
        #get DEM crs from DEM_ID 
        dem_id = df.DEM_ID[i]
        utm = "326" + dem_id[3:5]
        
        #reproject watershed polygon 
        merged_setcrs = merged.set_crs('EPSG:4326')
        merged_projected = merged_setcrs.to_crs('EPSG:'+ utm)
        
        dem_path = os.path.join("/nas/cee-water/cjgleason/wenwen/SARN_HMA/EarthDEM/Order_Downloads_2023-02-22-2002/earthdem", df.DEM_ID[i], df.DEM_ID[i]+"_dem.tif")
        raster = rxr.open_rasterio(dem_path)
        
        # Use geopandas/shapely to clip raster
        clipped_raster = raster.rio.clip([merged_projected.geometry[ID]])
        
        # Save clipped raster
        clipped_raster.rio.to_raster(str(ID)+ "_" + str(dem_id)+'.tif')
        
    #empty list of masked rasters to merge 
    raster_to_mosiac = []
    
    #append masked rasters to list 
    for i in list(df.index.values):
        dem_id = df.DEM_ID[i]
        folder = "/nas/cee-water/cjgleason/wenwen/SARN_HMA"
        path = os.path.join(folder, str(ID)+"_"+str(dem_id)+'.tif')
        raster = rio.open(path)
        raster_to_mosiac.append(raster)
    
    #merge list of rasters 
    mosaic, output = merge(raster_to_mosiac)
    
    #update mosaic metadata
    output_meta = raster.meta.copy()
    output_meta.update(
        {"driver": "GTiff",
            "height": mosaic.shape[1],
            "width": mosaic.shape[2],
            "transform": output,
        }
    )
    
    #write mosaic to folder 
    output_path = "/nas/cee-water/cjgleason/wenwen/SARN_HMA/mask_mosaics/"+str(ID)+".tif"
    with rio.open(output_path, "w", **output_meta) as m:
        m.write(mosaic)
        
    #delete intermediate tif files
    for i in list(df.index.values):
        dem_id = df.DEM_ID[i]
        folder = "/nas/cee-water/cjgleason/wenwen/SARN_HMA"
        path = os.path.join(folder, str(ID)+"_"+str(dem_id)+'.tif')
        os.remove(path)

In [ ]:
merged = hydrosheds_firstorder[hydrosheds_firstorder.COMID==45039031]

In [ ]:
merged.geometry[45039031]

In [ ]:
COMID_lst = [45017012]
for ID in COMID_lst: 
    df = intersection[intersection.COMID==ID]
    geom = df.geometry
    
    for i in list(df.index.values): 
        print(i)
        #get DEM crs from DEM_ID 
        dem_id = df.DEM_ID[i]
        print(dem_id)
        utm = "326" + dem_id[3:5]
        print(utm)
        
        #reproject watershed polygon 
        merged_setcrs = merged.set_crs('EPSG:4326')
        merged_projected = merged_setcrs.to_crs('EPSG:'+ utm)
        
        dem_path = os.path.join("/nas/cee-water/cjgleason/wenwen/SARN_HMA/EarthDEM/Order_Downloads_2023-02-22-2002/earthdem", df.DEM_ID[i], df.DEM_ID[i]+"_dem.tif")
        raster = rxr.open_rasterio(dem_path)
        
        # Use geopandas/shapely to clip raster
        clipped_raster = raster.rio.clip([merged_projected.geometry[ID]])
        
        # Save clipped raster
        clipped_raster.rio.to_raster(str(ID)+"_"+str(dem_id)+'test.tif')

In [ ]:
COMID_lst = [45039031]
for ID in COMID_lst: 
    df = intersection[intersection.COMID==ID]
    geom = df.geometry
        
    #empty list of masked rasters to merge 
    raster_to_mosiac = []
    
    #append masked rasters to list 
    for i in list(df.index.values):
        dem_id = df.DEM_ID[i]
        folder = "/nas/cee-water/cjgleason/wenwen/SARN_HMA"
        path = os.path.join(folder, str(ID)+"_"+str(dem_id)+'test.tif')
        raster = rio.open(path)
        raster_to_mosiac.append(raster)
    
    #merge list of rasters 
    mosaic, output = merge(raster_to_mosiac)
    
    #update mosaic metadata
    output_meta = raster.meta.copy()
    output_meta.update(
        {"driver": "GTiff",
            "height": mosaic.shape[1],
            "width": mosaic.shape[2],
            "transform": output,
        }
    )
    
    #write mosaic to folder 
    output_path = "/nas/cee-water/cjgleason/wenwen/SARN_HMA/mask_mosaics/"+str(ID)+"test.tif"
    with rio.open(output_path, "w", **output_meta) as m:
        m.write(mosaic)
        

In [ ]:
geom_projected

In [ ]:
geom_projected.to_file(str(ID)+".shp")

In [ ]:
COMID_lst = [45039031]
for ID in COMID_lst: 
    df = intersection[intersection.COMID==ID]
    geom = df.geometry
    
    for i in list(df.index.values): 
        
        #get DEM crs from DEM_ID 
        dem_id = df.DEM_ID[i]
        utm = "326" + dem_id[3:5]
        
        #reproject watershed polygon 
        geom_setcrs = geom.set_crs('EPSG:4326')
        geom_projected = geom_setcrs.to_crs('EPSG:'+ utm)
        
        dem_path = os.path.join("/nas/cee-water/cjgleason/wenwen/SARN_HMA/EarthDEM/Order_Downloads_2023-02-22-2002/earthdem", df.DEM_ID[i], df.DEM_ID[i]+"_dem.tif")
        raster = rxr.open_rasterio(dem_path)
        
        # Use geopandas/shapely to clip raster
        clipped_raster = raster.rio.clip([geom_projected.geometry[i]])
        
        # Save clipped raster
        clipped_raster.rio.to_raster(str(ID)+"_"+str(dem_id)+'.tif')
        
    #empty list of masked rasters to merge 
    raster_to_mosiac = []
    
    #append masked rasters to list 
    for i in list(df.index.values):
        dem_id = df.DEM_ID[i]
        folder = "/nas/cee-water/cjgleason/wenwen/SARN_HMA"
        path = os.path.join(folder, str(ID)+"_"+str(dem_id)+'.tif')
        raster = rio.open(path)
        raster_to_mosiac.append(raster)
    
    #merge list of rasters 
    mosaic, output = merge(raster_to_mosiac)
    
    #update mosaic metadata
    output_meta = raster.meta.copy()
    output_meta.update(
        {"driver": "GTiff",
            "height": mosaic.shape[1],
            "width": mosaic.shape[2],
            "transform": output,
        }
    )
    
    #write mosaic to folder 
    output_path = "/nas/cee-water/cjgleason/wenwen/SARN_HMA/mask_mosaics/"+str(ID)+".tif"
    with rio.open(output_path, "w", **output_meta) as m:
        m.write(mosaic)
        
    #delete intermediate tif files
    #for i in list(df.index.values):
        #dem_id = df.DEM_ID[i]
        #folder = "/nas/cee-water/cjgleason/wenwen/SARN_HMA"
        #path = os.path.join(folder, str(ID)+"_"+str(dem_id)+'.tif')
        #os.remove(path)

In [ ]:
# attempting to do the same without saving the intermediate masks as tifs
COMID_lst = [45035863]
for ID in COMID_lst: 
    df = intersection[intersection.COMID==ID]
    geom = df.geometry
    
    #empty list of masked rasters to merge 
    raster_to_mosiac = []
    
    for i in list(df.index.values): 
        
        #get DEM crs from DEM_ID 
        dem_id = df.DEM_ID[i]
        utm = "326" + dem_id[3:5]
        
        #reproject watershed polygon 
        geom_setcrs = geom.set_crs('EPSG:4326')
        geom_projected = geom_setcrs.to_crs('EPSG:'+ utm)
        
        dem_path = os.path.join("/nas/cee-water/cjgleason/wenwen/SARN_HMA/EarthDEM/Order_Downloads_2023-02-22-2002/earthdem", df.DEM_ID[i], df.DEM_ID[i]+"_dem.tif")
        raster = rxr.open_rasterio(dem_path)
        
        # Use geopandas/shapely to clip raster
        clipped_raster = raster.rio.clip([geom_projected.geometry[i]])
        
        raster_to_mosiac.append(clipped_raster)

    
    #append masked rasters to list 
    for i in list(df.index.values):
        dem_id = df.DEM_ID[i]
        folder = "/nas/cee-water/cjgleason/wenwen/SARN_HMA"
        path = os.path.join(folder, str(ID)+"_"+str(dem_id)+'.tif')
        raster = rio.open(path)
        raster_to_mosiac.append(raster)
    
    #merge list of rasters 
    mosaic, output = merge(raster_to_mosiac)
    
    #update mosaic metadata
    output_meta = raster.meta.copy()
    output_meta.update(
        {"driver": "GTiff",
            "height": mosaic.shape[1],
            "width": mosaic.shape[2],
            "transform": output,
        }
    )
    
    #write mosaic to folder 
    output_path = "/nas/cee-water/cjgleason/wenwen/SARN_HMA/mask_mosaics/"+str(ID)+".tif"
    with rio.open(output_path, "w", **output_meta) as m:
        m.write(mosaic)

In [ ]:
i=0 

In [ ]:
geoms = intersection["geometry"]

geoms = geoms.set_crs('EPSG:4326')
print(geoms.geometry)

#see how the polygon geometry values changes after we reproject 
geoms = geoms.to_crs('EPSG:32645')
print('\nReprojected: \n',geoms.geometry)


In [ ]:
dem_path = os.path.join("/nas/cee-water/cjgleason/wenwen/SARN_HMA/EarthDEM/Order_Downloads_2023-02-22-2002/earthdem", intersection.DEM_ID[i], intersection.DEM_ID[i]+"_dem.tif")
raster = rxr.open_rasterio(dem_path)
dem_path

In [ ]:
raster

In [ ]:
# Use geopandas/shapely to clip raster
clipped_raster = raster.rio.clip([geoms.geometry[0]])

# Save clipped raster
clipped_raster.rio.to_raster('riox_clip_int.tif')

In [ ]:
# Inspecting the output tif file
with rasterio.open('/nas/cee-water/cjgleason/wenwen/SARN_HMA/45035863_utm45n_30_03_2_2_2m_v1.0.tif', 'r') as src:
    img = src.read(1)
    img_met = src.meta
    
import matplotlib.pyplot as plt
plt.imshow(img)
img_met

In [ ]:
# Inspecting the output tif file
with rasterio.open('/nas/cee-water/cjgleason/wenwen/SARN_HMA/45035863_utm45n_31_03_1_2_2m_v1.0.tif', 'r') as src:
    img = src.read(1)
    img_met = src.meta
    
import matplotlib.pyplot as plt
plt.imshow(img)
img_met

In [ ]:
# Inspecting the output tif file
with rasterio.open('/nas/cee-water/cjgleason/wenwen/SARN_HMA/mask_mosaics/45035863.tif', 'r') as src:
    img = src.read(1)
    img_met = src.meta
    
import matplotlib.pyplot as plt
plt.imshow(img)
img_met

Working through the for loop: 

In [ ]:
len(COMID_lst)

In [ ]:
ID = 45035863

In [ ]:
df = intersection[intersection.COMID==ID]
df

In [ ]:
i=0
ID = 45035863

In [ ]:
df = intersection[intersection.COMID==ID]
geom = df.geometry

#get DEM crs from DEM_ID 
dem_id = df.DEM_ID[i]

utm = "326" + dem_id[3:5]

In [ ]:
dem_id

In [ ]:
#reproject watershed polygon 
geom_setcrs = geom.set_crs('EPSG:4326')
geom_projected = geom_setcrs.to_crs('EPSG:'+ utm)

In [ ]:
dem_path = os.path.join("/nas/cee-water/cjgleason/wenwen/SARN_HMA/EarthDEM/Order_Downloads_2023-02-22-2002/earthdem", df.DEM_ID[i], df.DEM_ID[i]+"_dem.tif")
dem_path

In [ ]:
raster = rxr.open_rasterio(dem_path)

In [ ]:
clipped_raster = raster.rio.clip([geom_projected.geometry[0]])

In [ ]:
clipped_raster

In [ ]:
type(clipped_raster)

In [ ]:
clipped_raster.rio.to_raster(str(ID)+"_"+str(dem_id)+'.tif')

In [ ]:
dem_id = df.DEM_ID[i]
folder = "/nas/cee-water/cjgleason/wenwen/SARN_HMA"
path = os.path.join(folder, str(ID)+"_"+str(dem_id)+'.tif')
raster = rio.open(path)
raster_to_mosiac.append(raster)

In [ ]:
type(raster)

In [ ]:
range(len(df))
for n in range(len(df)): 
    print(n)

In [ ]:
from rasterio.plot import show
from rasterio.merge import merge
import rasterio as rio
from pathlib import Path

In [ ]:
path = Path('data/')
Path('output').mkdir(parents=True, exist_ok=True)
output_path = 'output/mosaic_output.tif'

In [ ]:
raster_to_mosiac = []

In [ ]:
for i in list(df.index.values):
    dem_id = df.DEM_ID[i]
    folder = "/nas/cee-water/cjgleason/wenwen/SARN_HMA"
    path = os.path.join(folder, str(ID)+"_"+str(dem_id)+'.tif')
    raster = rio.open(path)
    raster_to_mosiac.append(raster)

In [ ]:
raster_to_mosiac

In [ ]:
mosaic, output = merge(raster_to_mosiac)

In [ ]:
output_meta = raster.meta.copy()
output_meta.update(
    {"driver": "GTiff",
        "height": mosaic.shape[1],
        "width": mosaic.shape[2],
        "transform": output,
    }
)

In [ ]:
with rio.open(output_path, "w", **output_meta) as m:
    m.write(mosaic)

In [ ]:
"/nas/cee-water/cjgleason/wenwen/SARN_HMA/mask_mosaics/"+str(ID)+"_mosaic_.tif"

In [ ]:
lst = intersection.COMID
COMID_lst = [*set(lst)]
i=0

In [ ]:
ID = 45035863
type(intersection[intersection.COMID==ID])

In [ ]:
df = intersection[intersection.COMID==ID]
geom = df.geometry

In [ ]:
df

In [ ]:
#get DEM crs from DEM_ID 
dem_id = df.DEM_ID[i]
utm = "326" + dem_id[3:5]
        
#reproject watershed polygon 
geom_setcrs = geom.set_crs('EPSG:4326')
geom_projected = geom_setcrs.to_crs('EPSG:'+ utm)

In [ ]:
geom_projected

In [ ]:
dem_path = os.path.join("/nas/cee-water/cjgleason/wenwen/SARN_HMA/EarthDEM/Order_Downloads_2023-02-22-2002/earthdem", df.DEM_ID[i], df.DEM_ID[i]+"_dem.tif")
raster = rxr.open_rasterio(dem_path)

In [ ]:
[geom_projected.geometry[0]]

In [ ]:
ID = 45017090
i=261
type(intersection[intersection.COMID==ID])

In [ ]:
df = intersection[intersection.COMID==ID]
geom = df.geometry

In [ ]:
df

In [ ]:
#get DEM crs from DEM_ID 
dem_id = df.DEM_ID[i]
utm = "326" + dem_id[3:5]
        
#reproject watershed polygon 
geom_setcrs = geom.set_crs('EPSG:4326')
geom_projected = geom_setcrs.to_crs('EPSG:'+ utm)

In [ ]:
geom_projected

In [ ]:
dem_path = os.path.join("/nas/cee-water/cjgleason/wenwen/SARN_HMA/EarthDEM/Order_Downloads_2023-02-22-2002/earthdem", df.DEM_ID[i], df.DEM_ID[i]+"_dem.tif")
raster = rxr.open_rasterio(dem_path)

In [ ]:
[geom_projected.geometry[261]]